## Column Generation Example

In [1]:
# Packages
import gurobipy as gp
import numpy as np
import random

In [25]:
# Data input
L = 120 # length of stock
N = 240 # number of stocks
M = 10  # number of customers

d = [10, 11, 11, 12, 12, 12, 10, 11, 12, 10]  # demand
l = [92, 59, 97, 32, 38, 55, 80, 75, 108, 57] # length

stocks = [('s' + str(i)) for i in range(1, N+1)]
customers = [('c' + str(i)) for i in range(1, M+1)]

demands = dict(zip(customers, d))
lengths = dict(zip(customers, l))

In [26]:
# MIP model (1)
m = gp.Model()

In [27]:
try:
    # Decision varibal
    y = m.addVars(stocks, vtype=gp.GRB.BINARY, name='y')
    x = m.addVars(stocks, customers, vtype=gp.GRB.INTEGER, name='x')

    # Objective function
    m.setObjective(gp.quicksum(1 * y[i] for i in stocks), gp.GRB.MINIMIZE)

    # Constraints
    for j in customers:
        m.addConstr((gp.quicksum(x[i, j] for i in stocks) >= demands[j]), name='constr1')
    for i in S:
        m.addConstr((gp.quicksum(lengths[j] * x[i, j] for j in customers) <= L * y[i]), name='constr2')
    
    # Optimize
    m.optimize()

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))
except AttributeError:
    print('Encountered an attribute error')


# Output Results
if m.status == gp.GRB.OPTIMAL:
    solution = m.getAttr('x', x)
    print(m.getObjective())

Optimize a model with 250 rows, 2640 columns and 5040 nonzeros
Variable types: 0 continuous, 2640 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve time: 0.04s
Presolved: 250 rows, 2640 columns, 5040 nonzeros
Variable types: 0 continuous, 2640 integer (1440 binary)

Root relaxation: objective 6.920000e+01, 489 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   69.20000    0   42          -   69.20000      -     -    0s
H    0     0                      75.0000000   69.20000  7.73%     -    0s
H    0     0                      74.0000000   69.20000  6.49%     -    0s
H    0     0                      72.0000000   69.20000  3.89%     -    0s
     0     0   69.20000    0   63   72.00000   69.20000  3.89%     -   

### Column Generation Algorithm

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [11]:
roll_width = np.array(120)
demand_width_array = np.array([92, 59, 97, 32, 38, 55, 80, 75, 108, 57])
demand_number_array = np.array([10, 11, 11, 12, 12, 12, 10, 11, 12, 10])
initial_cut_pattern = np.diag(np.floor(roll_width / demand_width_array))

In [12]:
initial_cut_pattern

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 3., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 3., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]])

In [13]:
initial_cut_pattern.shape

(10, 10)

In [14]:
def master_problem(columns, var_Type):
    '''
    Input Parameters
    columns: patterns set, 2-dimensional ndarray (m * n), each pattern represents a cut plan
    var_Type: integer or continuous 
    '''
    m = gp.Model()
    # variable 
    x = m.addVars(columns.shape[1], lb=0, vtype=var_Type)
    # objective
    m.setObjective(gp.quicksum(x), gp.GRB.MINIMIZE)
    # constraints
    for i in range(columns.shape[0]):
        m.addConstr(gp.quicksum(columns[i][j] * x[j] for j in range(columns.shape[1])) >= demand_number_array[i])
    # optimize model
    m.optimize()
    
    if var_Type == gp.GRB.CONTINUOUS:
        return np.array(m.getAttr('Pi', m.getConstrs()))
    else:
        obj = m.getObjective()
        return obj.getValue(), np.array(m.getAttr('X'))


def restricted_lp_master_problem(column):
    return master_problem(column, GRB.CONTINUOUS)


def restricted_ip_master_problem(column):
    return master_problem(column, GRB.INTEGER)


def knapsack_subproblem(kk):
    '''
    solve the knapsack subproblem
    Input: 
        kk: simplex multipliers, m-dimensional numpy array
    Output:
        flag_new_column: objective value
        new_column: new column can be added to master problem
    '''
    m = gp.Model()
    # variable
    a = m.addVars(kk.shape[0], lb=0, vtype=gp.GRB.INTEGER)
    # objective
    m.setObjective(gp.quicksum(kk[i] * a[i] for i in range(kk.shape[0])), GRB.MAXIMIZE)
    # constraint
    m.addConstr(gp.quicksum(demand_width_array[i] * a[i] for i in range(kk.shape[0])) <= roll_width)

    m.optimize()

    obj = m.getObjective()
    flag_new_column = obj.getValue()

    if flag_new_column > 1:
        new_column = m.getAttr('X')
    else:
        new_column = None
    return flag_new_column, new_column

In [15]:
flag_new_cut_pattern = 2
new_cut_pattern = None
cut_pattern = initial_cut_pattern

# Iterative add new column to master problem
while (flag_new_cut_pattern > 1):

    # add new cut pattern to the basis matrix (old cut pattern)
    if new_cut_pattern:
        cut_pattern = np.column_stack((cut_pattern, new_cut_pattern))
        print(cut_pattern)

    # get simplex multiplier
    kk = restricted_lp_master_problem(cut_pattern)

    # get sub-problem objective value and cut pattern
    flag_new_cut_pattern, new_cut_pattern = knapsack_subproblem(kk)

minimal_stock, optimal_number = restricted_ip_master_problem(cut_pattern)

print('************************************************')
print('parameter:')
print(f'roll_width: {roll_width}')
print(f'demand_width_array: {demand_width_array}')
print(f'demand_number_array: {demand_number_array}')
print('result:')
print(f'minimal_stock: {minimal_stock}')
print(f'cut_pattern: {cut_pattern}')
print(f'optimal_number: {optimal_number}')
print('************************************************')

Optimize a model with 10 rows, 10 columns and 10 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 10 rows and 10 columns
Presolve time: 0.04s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.8500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.10 seconds
Optimal objective  7.850000000e+01
Optimize a model with 1 rows, 10 columns and 10 nonzeros
Variable types: 0 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e+01, 1e+02]
  Objective range  [3e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 1.0000000
Presolve removed 0 rows and 7 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Found heuristic solution: objective 1.1666667
Variable types: 0